# Extraction des données du drone

Nous disposons des données de trois missions de vol du drone (longue, moyenne et courte), et pour chaque mission, quatre scénarios.
Nous nous intéressons principalement à trois fichiers pour l'extraction des données : *ulog_activator_0*, *ulog_mission_result_0* et *ulog_scheduler_0*.

Dans ce qui suis, nous choisissons donc la mission et le scénario que nous voulons analyser. Les fichiers CSV sont présents dans le dossier data, qui est dans le répertoire courant de ce notebook.

In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import csv
from utils.analyzer import *
import os

### Définition d'un extracteur de données

In [2]:
# Ce script était fait par l'équipe de kdbench, je l'utilise ici, 

def extractor(mission, scenario, job, tache):
    # Find out mission time boundaries
    
    items = Trace.from_mission_result(f"data_used/{mission}_mission/scenario{scenario}/ulog_mission_result_0.csv")
    lbound, rbound = items[0].ts, items[-1].ts
    
    # Load scheduler trace
    trace = Trace.from_scheduler(f"data_used/{mission}_mission/scenario{scenario}/ulog_scheduler_0.csv")
    
    # Load and merge activation trace in
    trace += Trace.from_activator(f"data_used/{mission}_mission/scenario{scenario}/ulog_activator_0.csv")
    
    # Restrict to time window enlarged by 1s on each side
    lbound - 1e6 < trace < rbound + 1e6
    
    # Create a trace analyzer object
    analyzer = JobAnalyzer(trace)
    
    return [getattr(j, job) for j in analyzer[MODULES.index(tache)]]

    

### Définition d'un exporteur de données sous format csv

In [3]:
def exporter(mission, scenario, tache, job , data):
    # ici on crée un dossier correspondant à la mission/scénario 
    output_dir = f"data_extracted/{mission}_mission/scenario{scenario}"
    os.makedirs(output_dir, exist_ok=True) 
    
    # on mets les résultas d'extraction dans le fichier correspondant
    with open(f"{output_dir}/{job}_{tache}.csv", mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['timestamp'])  
        for i in range(len(data)): 
            row = [data[i]]
            writer.writerow(row)

Ici, nous pouvons choisir la mission/scenario, ainsi que la tâche

In [4]:
ma_mission = 'short' # short, medium, long
mon_scenario = 1
ma_tache = 'pctl' #  (snsr', 'rctl', 'ekf2', 'actl', 'pctl', 'hte', 'fmgr', 'cmdr', 'navr')
###################
tasks_list = ['snsr', 'rctl', 'ekf2', 'actl', 'pctl', 'hte', 'fmgr', 'cmdr', 'navr'] # a ne pas changer


## Extraction des temps d'activation

### Extraction par tâche

Dans cette partie on effectue une extraction des temps d'activation pour une tâche spécéfique


In [5]:
res = extractor(ma_mission, mon_scenario, "xtime", ma_tache)

In [6]:
print(res[:10])
print("longueur de la liste: ", len(res))

[86, 28, 86, 85, 84, 85, 85, 85, 89, 28]
longueur de la liste:  1068


#### Exportation des données sous format csv

In [7]:
exporter(ma_mission, mon_scenario, ma_tache, "xtime", res)

### Extraction de tous les temps d'activation

En guise de facilitation de la tâche, nous allons procéder à l'extraction des temps d'activation des neuf tâches, puis mettre le tout dans une liste que nous trierons par la suite.

In [8]:
result_list = []

for tache in tasks_list:
    result_list += extractor(ma_mission, mon_scenario, "activation", tache)

result_list.sort()

In [9]:
print(result_list[:10])
print("longueur de la liste: ", len(result_list))

[24235406, 24236199, 24239298, 24241457, 24244457, 24245406, 24246209, 24247656, 24248457, 24254309]
longueur de la liste:  9431


#### Exportation des données sous format csv

In [10]:
exporter(ma_mission, mon_scenario, "all", "activation", result_list)

## Extraction des temps de fin de tâches

### Extraction par tâche

In [11]:
res = extractor(ma_mission, mon_scenario, "end", ma_tache)

In [12]:
print(res[:10])
print("longueur de la liste: ", len(res))

[24247758, 24262699, 24277757, 24292756, 24307755, 24322737, 24337756, 24352756, 24367773, 24382699]
longueur de la liste:  1068


#### Exportation des données sous format csv

In [13]:
exporter(ma_mission, mon_scenario, ma_tache, "end",  res)

### Extraction de tous les temps de fin

In [14]:
result_list = []

for tache in tasks_list:
    result_list += extractor(ma_mission, mon_scenario, "end", tache)

result_list.sort()

In [15]:
print(result_list[:10])
print("longueur de la liste: ", len(result_list))

[24235925, 24236447, 24239360, 24241640, 24244509, 24246194, 24246415, 24247758, 24248505, 24254371]
longueur de la liste:  9431


#### Exportation des données sous format csv

In [16]:
exporter(ma_mission, mon_scenario, "all", "end", result_list)

### Extraction des temps d'exécution

In [17]:
res = extractor(ma_mission, mon_scenario, "xtime", ma_tache)

In [18]:
print(res[:10])
print("longueur de la liste: ", len(res))

[86, 28, 86, 85, 84, 85, 85, 85, 89, 28]
longueur de la liste:  1068


In [19]:
exporter(ma_mission, mon_scenario, ma_tache, "xtime",  res)

### Extraction des start times

In [20]:
res = extractor(ma_mission, mon_scenario, "start", ma_tache)

In [21]:
print(res[:10])
print("longueur de la liste: ", len(res))

[24247672, 24262671, 24277671, 24292671, 24307671, 24322652, 24337671, 24352671, 24367684, 24382671]
longueur de la liste:  1068


In [22]:
exporter(ma_mission, mon_scenario, ma_tache, "start",  res)

## Extraction de tout les données possible

Le script suivant combine tout le code précédent pour extraire tous les données d'activation et de fin des tâches des trois missions avec leurs quatres scénarios

In [16]:
# Cette extraction peut être longue. à lancer si nécessaire.



# total = 2 * 3 * 4 * 9  # sert juste à montrer le progrès de l'extraction par la suite dans la boucle
# i = 1
# for job in ["activation", "end"]: # les deux job que nous voulons
#     for mission in ["short", "medium", "long"]:  #les 3 missions
#         for scenario in range(1,5): #les 4 scenario de chaque mission
#             all_tasks_results = [] # cette liste va servir pour sauvegarder les données extraites de toutes les tache d'un scénario dans un seul fichier
#             for tache in tasks_list: 
#                 prcntg = (i / total) * 100 #pourcentage de progression
#                 print(f"{prcntg:.1f} %", end='\r')
#                 res = extractor(mission, scenario, job, tache)
#                 res = [x for x in res if x is not None] #dans certains mission/scénario, l'extracteur laisse certaines cases vides ! 
#                 #exporter(mission, scenario, tache, job, res)
#                 all_tasks_results += res
#                 i += 1
#             exporter(mission, scenario, "all", job, sorted(all_tasks_results))     